# Importing packages

In [1]:
import utils
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/Users/colinwan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/colinwan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/colinwan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/colinwan/opt/anaconda3/lib/p

# Core Model Function

In [4]:
def CNN_Model(data, wv_from_bin, batch_size=512, kernel_size=5, vec_size=200, epochs=8):
    
    # Preparing Text Data and Label
    tweet, label = data['text'], data['sentiment']
    label = pd.concat([label==0, label==1, label==-1], axis=1).astype(int)

    # Tokenizing Text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweet)
    sequences = tokenizer.texts_to_sequences(tweet)
    max_length = max([len(s.split()) for s in tweet])

    # Preparing Embedding Dictionary
    word_index = tokenizer.word_index
    tweets_pad = pad_sequences(sequences, maxlen=max_length, padding='post')

    embedding_index = {}
    words = list(wv_from_bin.vocab.keys())
    curInd = 0
    for w in words:
        try:
            embedding_index[w] = wv_from_bin.word_vec(w)
            curInd += 1
        except KeyError:
            continue
            
    # Preparing Embedding Matrix
    embedding_matrix = np.zeros((len(word_index) + 1, vec_size))
    for word, i in word_index.items():
        # words.append(word)
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    # Preparing Embedding Layer
    embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            vec_size,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)
    
    # Construction CNN Model Structure
    model = Sequential()
    model.add(embedding_layer)
    model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Flatten())
    model.add(Dense(600))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
    print(model.summary())
    
    # Training Model
    model.fit(tweets_pad, label, batch_size=batch_size, epochs=epochs, validation_split=0.1, shuffle=True, callbacks=[reduce_lr])
    
    # Saving Trained Model
    model.save("/Users/colinwan/Desktop/DataFest2020/Models/CNN.h5")
    
    return model

In [5]:
if __name__ == "__main__":
    # Load GloVe Vectors
    wv_from_bin = utils.load_embedding_model(200)
    # Load data file
    data = pd.read_csv('/Users/colinwan/Desktop/DataFest2020/Training Folder/Sentiment.csv')
    data = data[['text','sentiment']]
    # Replayce word with class number
    data = data.replace('Neutral', 0)
    data = data.replace('Positive', 1)
    data = data.replace('Negative', -1)
    data = utils.clean_text(data)
    # Filter out none alphabet character
    data['text'] = data['text'].str.replace('[^A-Za-z ]+', '')
    # Run model
    CNN_M = CNN_Model(data, wv_from_bin, epochs=32)

Loaded vocab size 400000


/Users/colinwan/Desktop/DataFest2020/Models/ipython/utils.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = " ".join([word for word in df['text'][i].split()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 200)           2492800   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 25, 300)           300300    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 21, 150)           225150    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 17, 75)            56325     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1275)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 600)               765600    
_________________________________________________________________
activation_3 (Activation)    (None, 600)              